In [1]:
%matplotlib inline
import os,sys
pwd = os.getcwd()
parent = '/'.join(pwd.split('/')[:-1])
sys.path.insert(0,parent)
os.chdir(parent)
#%%
print('-'*30)
print(os.getcwd())
print('-'*30)
#%%
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from core.DAZLE import DAZLE
from core.CUBDataLoader import CUBDataLoader
from core.helper_func import eval_zs_gzsl,visualize_attention,eval_zs_gzsl#,get_attribute_attention_stats
from global_setting import NFS_path
import importlib
import pdb
import numpy as np

------------------------------
/home/project_amadeus/home/hbdat/[RELEASE]_DenseAttentionZSL
------------------------------


In [2]:
idx_GPU = 4
device = torch.device("cuda:{}".format(idx_GPU) if torch.cuda.is_available() else "cpu")

In [3]:
torch.backends.cudnn.benchmark = True

In [4]:
dataloader = CUBDataLoader(NFS_path,device,is_unsupervised_attr=False,is_balance=False)

/home/project_amadeus/mnt/raptor/hbdat/Attention_over_attention/
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
CUB
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
_____
/home/project_amadeus/mnt/raptor/hbdat/Attention_over_attention/data/CUB/feature_map_ResNet_101_CUB.hdf5
Expert Attr
Finish loading data in  61.433818


In [5]:
dataloader.augment_img_path()

In [6]:
def get_lr(optimizer):
    lr = []
    for param_group in optimizer.param_groups:
        lr.append(param_group['lr'])
    return lr

In [7]:
seed = 214#215#
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

batch_size = 50
nepoches = 20
niters = dataloader.ntrain * nepoches//batch_size
dim_f = 2048
dim_v = 300
init_w2v_att = dataloader.w2v_att
att = dataloader.att
normalize_att = dataloader.normalize_att

trainable_w2v = True
lambda_ = 0.1#0.1
bias = 0
prob_prune = 0
uniform_att_1 = False
uniform_att_2 = False

seenclass = dataloader.seenclasses
unseenclass = dataloader.unseenclasses
desired_mass = 1
report_interval = niters//nepoches

model = DAZLE(dim_f,dim_v,init_w2v_att,att,normalize_att,
            seenclass,unseenclass,
            lambda_,
            trainable_w2v,normalize_V=False,normalize_F=True,is_conservative=True,
            uniform_att_1=uniform_att_1,uniform_att_2=uniform_att_2,
            prob_prune=prob_prune,desired_mass=desired_mass, is_conv=False,
            is_bias=True)
model.to(device)

setup = {'pmp':{'init_lambda':0.1,'final_lambda':0.1,'phase':0.8},
         'desired_mass':{'init_lambda':-1,'final_lambda':-1,'phase':0.8}}
print(setup)
#scheduler = Scheduler(model,niters,batch_size,report_interval,setup)

params_to_update = []
params_names = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        params_names.append(name)
        print("\t",name)
#%%
lr = 0.0001
weight_decay = 0.0001#0.000#0.#
momentum = 0.9#0.#
#%%
lr_seperator = 1
lr_factor = 1
print('default lr {} {}x lr {}'.format(params_names[:lr_seperator],lr_factor,params_names[lr_seperator:]))
optimizer  = optim.RMSprop( params_to_update ,lr=lr,weight_decay=weight_decay, momentum=momentum)

print('-'*30)
print('learing rate {}'.format(lr))
print('trainable V {}'.format(trainable_w2v))
print('lambda_ {}'.format(lambda_))
print('optimized seen only')
print('optimizer: RMSProp with momentum = {} and weight_decay = {}'.format(momentum,weight_decay))
print('-'*30)

/home/project_amadeus/home/hbdat/[RELEASE]_DenseAttentionZSL/core/DAZLE.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.init_w2v_att = F.normalize(torch.tensor(init_w2v_att))
/home/project_amadeus/home/hbdat/[RELEASE]_DenseAttentionZSL/core/DAZLE.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.att = nn.Parameter(F.normalize(torch.tensor(att)),requires_grad = False)


------------------------------
Configuration
loss_type CE
no constraint V
normalize F
training to exclude unseen class [seen upperbound]
Init word2vec
Linear model
loss_att BCEWithLogitsLoss()
Bilinear attention module
******************************
Measure w2v deviation
new Laplacian smoothing with desire mass 1 4
Compute Pruning loss 0
Add one smoothing
Second layer attenion conditioned on image features
------------------------------
No sigmoid on attr score
{'pmp': {'init_lambda': 0.1, 'final_lambda': 0.1, 'phase': 0.8}, 'desired_mass': {'init_lambda': -1, 'final_lambda': -1, 'phase': 0.8}}
	 V
	 W_1
	 W_2
	 W_3
default lr ['V'] 1x lr ['W_1', 'W_2', 'W_3']
------------------------------
learing rate 0.0001
trainable V True
lambda_ 0.1
optimized seen only
optimizer: RMSProp with momentum = 0.9 and weight_decay = 0.0001
------------------------------


In [8]:
best_performance = [0,0,0,0]
for i in range(0,niters):
    model.train()
    optimizer.zero_grad()
    
    batch_label, batch_feature, batch_att = dataloader.next_batch(batch_size)
    out_package = model(batch_feature)
    
    in_package = out_package
    in_package['batch_label'] = batch_label
    
    out_package=model.compute_loss(in_package)
    loss,loss_CE,loss_cal = out_package['loss'],out_package['loss_CE'],out_package['loss_cal']
    
    loss.backward()
    optimizer.step()
    if i%report_interval==0:
        print('-'*30)
        acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader,model,device,bias_seen=-bias,bias_unseen=bias)
        
        if H > best_performance[2]:
            best_performance = [acc_seen, acc_novel, H, acc_zs]
        stats_package = {'iter':i, 'loss':loss.item(), 'loss_CE':loss_CE.item(),
                         'loss_cal': loss_cal.item(),
                         'acc_seen':best_performance[0], 'acc_novel':best_performance[1], 'H':best_performance[2], 'acc_zs':best_performance[3]}
        
        print(stats_package)

------------------------------
bias_seen 0 bias_unseen 0
{'iter': 0, 'loss': 5.339369297027588, 'loss_CE': 5.305324554443359, 'loss_cal': 0.34044915437698364, 'acc_seen': 0, 'acc_novel': 0, 'H': 0, 'acc_zs': 0}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 141, 'loss': 1.6474179029464722, 'loss_CE': 1.5508091449737549, 'loss_cal': 0.9660871028900146, 'acc_seen': 0.32329174876213074, 'acc_novel': 0.4783506989479065, 'H': 0.3858249633018277, 'acc_zs': 0.5206593871116638}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 282, 'loss': 1.5724619626998901, 'loss_CE': 1.4667860269546509, 'loss_cal': 1.0567588806152344, 'acc_seen': 0.44570451974868774, 'acc_novel': 0.5117195248603821, 'H': 0.4764361337239027, 'acc_zs': 0.579412579536438}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 423, 'loss': 1.3222812414169312, 'loss_CE': 1.2042889595031738, 'loss_cal': 1.179922342300415, 'acc_seen': 0.5000437498092651, 'acc_novel': 0.537502646446228,